In [1]:
from bids.grabbids import BIDSLayout

In [2]:
layout = BIDSLayout("/data/ds102/")

In [3]:
layout.get_subjects()

['01', '02', '03', '04', '05']

In [4]:
layout.get_modalities()

['anat', 'func']

In [5]:
layout.get_types()

['bold', 'events', 'T1w']

In [6]:
layout.get_tasks()

['flanker']

In [7]:
layout.get(subject='01')

[File(filename='/data/ds102/sub-01/anat/sub-01_T1w.nii.gz', type='T1w', modality='anat', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', type='bold', task='flanker', run='1', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-1_events.tsv', type='events', task='flanker', run='1', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz', type='bold', task='flanker', run='2', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-2_events.tsv', type='events', task='flanker', run='2', modality='func', subject='01')]

In [8]:
layout.get(subject='01', type='bold', extensions=['nii', 'nii.gz'])

[File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', type='bold', task='flanker', run='1', modality='func', subject='01'),
 File(filename='/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz', type='bold', task='flanker', run='2', modality='func', subject='01')]

In [9]:
[f.filename for f in layout.get(subject='01', type='T1w', extensions=['nii', 'nii.gz'])]

['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz']

In [35]:
def get_niftis(subject_id, data_dir):
    from bids.grabbids import BIDSLayout
    layout = BIDSLayout(data_dir)
    bolds = [f.filename for f in layout.get(subject=subject_id, type='bold', extensions=['nii', 'nii.gz'])]
    T1ws = [f.filename for f in layout.get(subject=subject_id, type='T1w', extensions=['nii', 'nii.gz'])]
    return bolds, T1ws

In [36]:
get_niftis('01', '/data/ds102')

(['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz',
  '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz'],
 ['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz'])

In [37]:
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import IdentityInterface, Function

In [38]:
BIDSDataGrabber = Node(Function(function=get_niftis, input_names=["subject_id",
                                       "data_dir"],
                                   output_names=["bolds", 
                                        "T1ws"]), name="BIDSDataGrabber")
BIDSDataGrabber.inputs.data_dir = "/data/ds102"

In [39]:
BIDSDataGrabber.inputs.subject_id='01'
res = BIDSDataGrabber.run()
res.outputs

170324-02:58:57,265 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmp1XHeba/BIDSDataGrabber



T1ws = ['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz']
bolds = ['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz']

In [40]:
def printMe(paths):
    print("\n\nanalyzing " + str(paths) + "\n\n")
    
analyzeBOLD = Node(Function(function=printMe, input_names=["paths"],
                                   output_names=[]), name="analyzeBOLD")

analyzeT1w = Node(Function(function=printMe, input_names=["paths"],
                                   output_names=[]), name="analyzeT1w")


In [41]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD, "paths")
wf.connect(BIDSDataGrabber, "T1ws", analyzeT1w, "paths")
wf.run()

170324-02:58:59,104 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170324-02:58:59,213 workflow INFO:
	 Running serially.
170324-02:58:59,215 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmpNh8bYU/bids_demo/BIDSDataGrabber
170324-02:58:59,276 workflow INFO:
	 Executing node analyzeT1w in dir: /tmp/tmpNH7IN8/bids_demo/analyzeT1w


analyzing ['/data/ds102/sub-01/anat/sub-01_T1w.nii.gz']


170324-02:58:59,302 workflow INFO:
	 Executing node analyzeBOLD in dir: /tmp/tmpHF95Mm/bids_demo/analyzeBOLD


analyzing ['/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-01/func/sub-01_task-flanker_run-2_bold.nii.gz']




In [42]:
BIDSDataGrabber.iterables = ('subject_id', layout.get_subjects())
wf.run()

170324-02:59:05,805 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170324-02:59:05,923 workflow INFO:
	 Running serially.
170324-02:59:05,926 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a2 in dir: /tmp/tmpPju_aG/bids_demo/_subject_id_03/BIDSDataGrabber
170324-02:59:05,989 workflow INFO:
	 Executing node analyzeT1w.a2 in dir: /tmp/tmph4YmXJ/bids_demo/_subject_id_03/analyzeT1w


analyzing ['/data/ds102/sub-03/anat/sub-03_T1w.nii.gz']


170324-02:59:06,15 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a1 in dir: /tmp/tmpEVftZ4/bids_demo/_subject_id_02/BIDSDataGrabber
170324-02:59:06,67 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a3 in dir: /tmp/tmp58bQbR/bids_demo/_subject_id_04/BIDSDataGrabber
170324-02:59:06,129 workflow INFO:
	 Executing node analyzeBOLD.a3 in dir: /tmp/tmp76029c/bids_demo/_subject_id_04/analyzeBOLD


analyzing ['/data/ds102/sub-04/func/sub-04_task-flanker_run-1_bold.nii.gz', '/data/ds102/sub-04/func/sub-04_task-flanke

In [43]:
layout.get_metadata('/data/ds102/sub-01/func/sub-01_task-flanker_run-1_bold.nii.gz')

{u'CogAtlasID': u'tsk_4a57abb949a4f',
 u'EchoTime': 0.03,
 u'FlipAngle': 80.0,
 u'MRAcquisitionType': u'2D',
 u'MagneticFieldStrength': 3.0,
 u'Manufacturer': u'Siemens',
 u'ManufacturerModelName': u'Allegra',
 u'RepetitionTime': 2.0,
 u'ScanningSequence': u'Echo Planar',
 u'TaskDescription': u'On each trial (inter-trial interval (ITI) varied between 8 s and 14 s; mean ITI=12 s),participants used one of two buttons on a response pad to indicate the direction of a central arrow in an array of 5 arrows. In congruent trials the flanking arrows pointed in the same direction as the central arrow (e.g., < < < < <), while in more demanding incongruent trials the flanking arrows pointed in the opposite direction (e.g., < < > < <). Subjects performed two 5-minute blocks, each containing 12 congruent and 12 incongruent trials, presented in a pseudorandom order.',
 u'TaskName': u'Flanker'}

In [44]:
def printMetadata(path, data_dir):
    from bids.grabbids import BIDSLayout
    layout = BIDSLayout(data_dir)
    print("\n\nanalyzing " + path + "\nTR: "+ str(layout.get_metadata(path)["RepetitionTime"]) + "\n\n")
    
analyzeBOLD2 = MapNode(Function(function=printMetadata, input_names=["path", "data_dir"],
                             output_names=[]), name="analyzeBOLD2", iterfield="path")
analyzeBOLD2.inputs.data_dir = "/data/ds102/"

In [45]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD2, "path")
wf.connect(BIDSDataGrabber, "T1ws", analyzeT1w, "paths")
wf.run()

170324-02:59:09,793 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170324-02:59:09,919 workflow INFO:
	 Running serially.
170324-02:59:09,922 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a1 in dir: /tmp/tmpxXSj_M/bids_demo/_subject_id_02/BIDSDataGrabber
170324-02:59:09,977 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a4 in dir: /tmp/tmpBw6kUG/bids_demo/_subject_id_05/BIDSDataGrabber
170324-02:59:10,56 workflow INFO:
	 Executing node analyzeT1w.a4 in dir: /tmp/tmpkQ3sOk/bids_demo/_subject_id_05/analyzeT1w


analyzing ['/data/ds102/sub-05/anat/sub-05_T1w.nii.gz']


170324-02:59:10,80 workflow INFO:
	 Executing node analyzeT1w.a1 in dir: /tmp/tmpyh5HdP/bids_demo/_subject_id_02/analyzeT1w


analyzing ['/data/ds102/sub-02/anat/sub-02_T1w.nii.gz']


170324-02:59:10,119 workflow INFO:
	 Executing node analyzeBOLD2.a4 in dir: /tmp/tmp74V8Ry/bids_demo/_subject_id_05/analyzeBOLD2
170324-02:59:10,138 workflow INFO:
	 Executing node _analyzeBOLD20 in di